In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule
import pytorch_lightning as pl

import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from src.models import *
# from ilan_src.models import *
from src.dataloader import *
from src.utils import *
from src.evaluation import *

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import pickle

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [3]:
import sys

## Set up dataset - my way

In [4]:
members = 10
zero_noise = False

In [5]:
DATADRIVE = '/home/jupyter/data/'

In [6]:
# ds_train = TiggeMRMSDataset(
#     tigge_dir=f'{DATADRIVE}/tigge/32km/',
#     tigge_vars=['total_precipitation_ens10'],
#     mrms_dir=f'{DATADRIVE}/mrms/4km/RadarOnly_QPE_06H/',
#     rq_fn=f'{DATADRIVE}/mrms/4km/RadarQuality.nc',
#     data_period=('2018-01', '2019-12'),
#     val_days=5,
#     split='train',
#     tp_log=0.01,
#     ensemble_mode='random',
#     idx_stride=16
# )

In [7]:
# !mkdir tmp

In [8]:
# ds_train.mins.to_netcdf('tmp/mins1.nc')
# ds_train.maxs.to_netcdf('tmp/maxs1.nc')

In [9]:
mins = xr.open_dataset('tmp/mins1.nc')
maxs = xr.open_dataset('tmp/maxs1.nc')

In [10]:
ds_test = TiggeMRMSDataset(
    tigge_dir=f'{DATADRIVE}/tigge/32km/',
    tigge_vars=['total_precipitation_ens10'],
    mrms_dir=f'{DATADRIVE}/mrms/4km/RadarOnly_QPE_06H/',
#     rq_fn=f'{DATADRIVE}/mrms/4km/RadarQuality.nc',
    data_period=('2020-01', '2020-12'),
    first_days=5,
    tp_log=0.01,
    mins=mins,
    maxs=maxs,
    ensemble_mode='random',
    idx_stride=16
)

/opt/conda/envs/ilan/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Loading data


In [ ]:
# For ens_tp with stuff
ds_test_pad = TiggeMRMSDataset(
    tigge_dir=f'{DATADRIVE}/tigge/32km/',
    tigge_vars=['total_precipitation_ens10', 'total_column_water_ens10'],
    mrms_dir=f'{DATADRIVE}/mrms/4km/RadarOnly_QPE_06H/',
#     rq_fn=f'{DATADRIVE}/mrms/4km/RadarQuality.nc',
    data_period=('2020-01', '2020-12'),
    first_days=5,
    tp_log=0.01,
    mins=mins,
    maxs=maxs,
    ensemble_mode='random',
    idx_stride=16,
    pad_tigge=10,
    pad_tigge_channel=True,
)

/opt/conda/envs/ilan/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/opt/conda/envs/ilan/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Loading data
> /home/jupyter/nwp-downscale/notebooks/stephan_notebooks/src/dataloader.py(93)__init__()
     91         import pdb
     92         pdb.set_trace()
---> 93         self.tigge.load(); self.mrms.load()   # Load datasets into RAM
     94         if tp_log:
     95             self.tigge['tp'] = log_trans(self.tigge['tp'], tp_log)



ipdb>  self.tigge


<xarray.Dataset>
Dimensions:     (lat: 93, lon: 172, member: 10, valid_time: 101)
Coordinates:
    lead_time   timedelta64[ns] 12:00:00
    init_time   (valid_time) datetime64[ns] 2020-01-01 ... 2020-11-30T12:00:00
  * member      (member) int32 0 1 2 3 4 5 6 7 8 9
  * lon         (lon) float64 235.2 235.5 235.8 236.2 ... 289.3 289.6 289.9
  * lat         (lat) float64 49.68 49.36 49.04 48.72 ... 21.2 20.88 20.56 20.24
  * valid_time  (valid_time) datetime64[ns] 2020-01-01T12:00:00 ... 2020-12-01
Data variables:
    tp          (member, valid_time, lat, lon) float32 0.205 0.798 ... 0.0 0.0
    tcw         (member, valid_time, lat, lon) float32 13.05 13.78 ... 23.31
Attributes:
    regrid_method:  bilinear


ipdb>  n


> /home/jupyter/nwp-downscale/notebooks/stephan_notebooks/src/dataloader.py(94)__init__()
     92         pdb.set_trace()
     93         self.tigge.load(); self.mrms.load()   # Load datasets into RAM
---> 94         if tp_log:
     95             self.tigge['tp'] = log_trans(self.tigge['tp'], tp_log)
     96             if self.cat_bins is None:   # No log transform for categorical output



In [13]:
ds_test_pad.tigge

<xarray.DataArray (variable: 1, member: 10, valid_time: 101, lat: 93, lon: 172)>
array([[[[[3.01053647e-01, 4.30970623e-01, 5.74420528e-01, ...,
           4.07775744e-01, 4.03023890e-01, 3.98107896e-01],
          [3.50360558e-01, 2.98886125e-01, 1.36070486e-01, ...,
           3.84204037e-01, 3.81259676e-01, 4.00963425e-01],
          [6.29287130e-01, 5.98463392e-01, 4.89741797e-01, ...,
           3.65489415e-01, 3.71376218e-01, 3.85884938e-01],
          ...,
          [0.00000000e+00, 0.00000000e+00, 2.32960875e-02, ...,
           4.33855303e-02, 7.74335538e-04, 2.79797671e-03],
          [5.57682546e-02, 9.93955513e-02, 1.91622802e-01, ...,
           0.00000000e+00, 1.24615663e-02, 4.71934326e-02],
          [1.11713518e-01, 1.68800772e-01, 1.86983340e-01, ...,
           2.83188763e-01, 2.58380267e-01, 3.11095490e-01]],

         [[4.67681586e-01, 3.09886144e-01, 3.58872728e-01, ...,
           2.70211285e-01, 2.51587437e-01, 2.33679436e-01],
          [2.17803054e-01, 1.11798255e-01, 2.95733102e-02, ...,
           2.97110918e-01, 2.72448244e-01, 2.30026949e-01],
          [4.62480264e-01, 3.71839116e-01, 2.67128682e-01, ...,
           3.18388955e-01, 2.94163562e-01, 2.44502470e-01],
...
          [1.36581859e-02, 2.67711691e-03, 1.58821370e-02, ...,
           4.65337618e-01, 4.74839874e-01, 4.81428487e-01],
          [5.12692571e-03, 0.00000000e+00, 2.35492240e-02, ...,
           4.97496663e-01, 4.66686727e-01, 4.20776149e-01],
          [0.00000000e+00, 0.00000000e+00, 1.67929464e-01, ...,
           4.39200834e-01, 4.23750479e-01, 4.30086190e-01]],

         [[4.57715637e-02, 1.64082959e-02, 1.57480472e-01, ...,
           4.20917784e-01, 4.20543460e-01, 4.32545872e-01],
          [1.15769199e-01, 3.10847969e-01, 3.15031832e-01, ...,
           4.16436239e-01, 4.28971173e-01, 4.48385757e-01],
          [7.41873945e-02, 6.16905950e-02, 5.31865903e-02, ...,
           4.45216919e-01, 4.71863953e-01, 4.98515993e-01],
          ...,
          [8.82416384e-02, 7.45681052e-03, 0.00000000e+00, ...,
           1.12155828e-02, 9.44849380e-03, 0.00000000e+00],
          [1.24867302e-01, 5.17970068e-02, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [1.27003755e-01, 9.73690793e-02, 6.68321427e-02, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]]]])
Coordinates:
    lead_time   timedelta64[ns] 12:00:00
    init_time   (valid_time) datetime64[ns] 2020-01-01 ... 2020-11-30T12:00:00
  * member      (member) int32 0 1 2 3 4 5 6 7 8 9
  * lon         (lon) float64 235.2 235.5 235.8 236.2 ... 289.3 289.6 289.9
  * lat         (lat) float64 49.68 49.36 49.04 48.72 ... 21.2 20.88 20.56 20.24
  * valid_time  (valid_time) datetime64[ns] 2020-01-01T12:00:00 ... 2020-12-01
  * variable    (variable) <U2 'tp'

In [11]:
ds_test_pad[0][0].shape

(2, 16, 16)

## Load models

### Model 1: single_forecast_tp_pure_sr_pretraining

In [15]:
name='single_forecast_tp_pure_sr_pretraining'

In [16]:
zero_noise=True

In [17]:
model_dir = '/home/jupyter/data/saved_models/leingan/single_forecast_tp_pure_sr_pretraining/0'

In [18]:
sys.path.append(model_dir)

In [19]:
gan = BaseGAN2.load_from_checkpoint(
    f"{model_dir}/epoch=199-step=133999.ckpt")

In [20]:
model = gan.gen
model = model.to(device)
model.train(False);

## Model 2: ens_mean_L1_weighted_gen_loss

In [11]:
name='ens_mean_L1_weighted_gen_loss'

In [15]:
model_dir = '/home/jupyter/data/saved_models/leingan/ens10_tp/random/ens_mean_L1_weighted_gen_loss/3'

In [16]:
sys.path.append(model_dir)

In [17]:
gan = BaseGAN2.load_from_checkpoint(
    f"{model_dir}/epoch=349-step=234499.ckpt")

In [18]:
model = gan.gen
model = model.to(device)
model.train(False);

### Model 3: ens10_tp_and_added_vars_TCW_broadfield_channel

In [12]:
name='ens10_tp_and_added_vars_TCW_broadfield_channel'

In [35]:
ds_test = ds_test_pad

In [14]:
model_dir = '/home/jupyter/data/saved_models/leingan/ens10_tp_and_added_vars_TCW_broadfield_channel/15'

In [15]:
sys.path.append(model_dir)

In [16]:
gan = BaseGAN2.load_from_checkpoint(
    f"{model_dir}/epoch=499-step=258499.ckpt")

In [17]:
model = gan.gen
model = model.to(device)
model.train(False);

## Full field eval

In [18]:
def create_valid_predictions(model, ds_valid, member_idx=None, zero_noise=False):
    # Get predictions for full field
    preds = []
    for t in tqdm(range(len(ds_valid.tigge.valid_time))):
        X, y = ds_valid.return_full_array(t, member_idx=member_idx)
        noise = torch.randn(1, X.shape[0], X.shape[1], X.shape[2]).to(device)
        if zero_noise:
            noise *= 0
        pred = model(torch.FloatTensor(X[None]).to(device), noise).to('cpu').detach().numpy()[0, 0]
        preds.append(pred)
    preds = np.array(preds)
    
    # Unscale
    preds = preds * (ds_valid.maxs.tp.values - ds_valid.mins.tp.values) + ds_valid.mins.tp.values
    
    # Un-log
    if ds_valid.tp_log:
        preds = log_retrans(preds, ds_valid.tp_log)
    
    # Convert to xarray
    preds = xr.DataArray(
        preds,
        dims=['valid_time', 'lat', 'lon'],
        coords={
            'valid_time': ds_valid.tigge.valid_time,
            'lat': ds_valid.mrms.lat.isel(
                lat=slice(ds_valid.pad_mrms, ds_valid.pad_mrms+preds.shape[1])
            ),
            'lon': ds_valid.mrms.lon.isel(
                lon=slice(ds_valid.pad_mrms, ds_valid.pad_mrms+preds.shape[2])
            )
        },
        name='tp'
    )
    return preds

In [19]:
def create_stitched_predictions(model, ds_test, member_idx, zero_noise=False):
    preds = ds_test.mrms.copy(True) * np.NaN
    for idx in tqdm(range(len(ds_test.idxs))):
        time_idx, lat_idx, lon_idx = ds_test.idxs[idx]
        lat_slice = slice(lat_idx * ds_test.ratio, lat_idx * ds_test.ratio + ds_test.patch_mrms)
        lon_slice = slice(lon_idx * ds_test.ratio, lon_idx * ds_test.ratio + ds_test.patch_mrms)

        X, y = ds_test.__getitem__(idx, member_idx=member_idx)

        noise = torch.randn(1, X.shape[0], X.shape[1], X.shape[2]).to(device)
        if zero_noise:
            noise *= 0
        p = model(torch.FloatTensor(X[None]).to(device), noise).to('cpu').detach().numpy()[0, 0]

        preds[time_idx, lat_slice, lon_slice] = p
    # Unscale
    preds = preds * (ds_test.maxs.tp.values - ds_test.mins.tp.values) + ds_test.mins.tp.values

    # Un-log
    if ds_test.tp_log:
        preds = log_retrans(preds, ds_test.tp_log)
    preds = preds.rename({'time': 'valid_time'})
    return preds

In [20]:
def create_valid_ensemble(model, ds_valid, nens, stitched=False, zero_noise=False):
    """Wrapper to create ensemble"""
    if stitched:
        fn = create_stitched_predictions
    else:
        fn = create_valid_predictions
    preds = [fn(model, ds_valid, member_idx=member_idx, zero_noise=zero_noise) for member_idx in range(nens)]
    return xr.concat(preds, 'member')

In [22]:
# ens_pred = create_valid_ensemble(model, ds_test, members, zero_noise=zero_noise)

In [36]:
X, y = ds_test[0]

In [37]:
X.shape

(2, 16, 16)

In [27]:
model

LeinGen2(
  (embed): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): LeinResBlock(
      (shortcut): Sequential(
        (0): ConvBlock(
          (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (convblock1): ConvBlock(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (convblock2): ConvBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): LeinResBlock(
      (shortcut): Sequential(
        (0): ConvBlock(
          (conv): Conv2d(128, 255, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (convblock1): ConvBlock(
        (conv): Conv2d(128, 255, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (convblock2): ConvBlock(
        (conv): Conv2d(255, 255, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (process): Sequential(
    (0): LeinRes

In [32]:
ens_pred_stitched = create_valid_ensemble(model, ds_test, members, stitched=True, zero_noise=zero_noise)

  0%|          | 0/3636 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 2, 16, 16] to have 3 channels, but got 2 channels instead

In [ ]:
ens_pred.to_netcdf(f'tmp/ens_pred_{name}.nc')
ens_pred_stitched.to_netcdf(f'tmp/ens_pred_stitched_{name}.nc')

In [ ]:
ens_pred.isel(valid_time=1).plot(vmin=0, vmax=20, cmap='gist_ncar_r', col='member')

## Get ground truth

In [ ]:
mrms = ds_test.mrms.rename(
    {'time': 'valid_time'}) * ds_test.maxs.tp.values
mrms = log_retrans(mrms, ds_test.tp_log)

In [ ]:
mrms.to_netcdf('tmp/mrms.nc')

## Get interpolation baseline

In [ ]:
tigge = ds_test.tigge.isel(variable=0) * ds_test.maxs.tp.values
tigge = log_retrans(tigge, ds_test.tp_log)

In [ ]:
interp = tigge.interp_like(mrms, method='linear')

In [ ]:
interp.to_netcdf('tmp/interp_ens.nc')

### HREF

In [ ]:
href = xr.open_mfdataset('/home/jupyter/data/hrefv2//4km/total_precipitation/2020*.nc')

In [ ]:
href = href.tp.diff('lead_time').sel(lead_time=np.timedelta64(12, 'h'))

In [ ]:
href['valid_time'] = href.init_time + href.lead_time
href = href.swap_dims({'init_time': 'valid_time'})

In [ ]:
href = href.assign_coords({'lat': interp.lat.values, 'lon': interp.lon.values})

In [ ]:
overlap_times = np.intersect1d(interp.valid_time, href.valid_time)

In [ ]:
href = href.sel(valid_time=overlap_times)

In [ ]:
href.load();

In [ ]:
href.to_netcdf('tmp/href.nc')

# Old

## Get mask

In [ ]:
ds = xr.open_dataset(
    '/home/jupyter/data/hrrr/raw/total_precipitation/20180215_00.nc')

In [ ]:
from src.regrid import *

In [ ]:
ds_regridded = regrid(ds, 4, lons=(235, 290), lats=(50, 20))

In [ ]:
hrrr_mask = np.isfinite(ds_regridded).tp.isel(init_time=0, lead_time=0)

In [ ]:
rq.plot(vmin=0, vmax=1)

In [ ]:
(rq>0.3).plot(vmin=0, vmax=1)

In [ ]:
mrms_mask.plot(vmin=0, vmax=1)

In [ ]:
rq = xr.open_dataarray(f'{DATADRIVE}/mrms/4km/RadarQuality.nc')
mrms_mask = rq>-1
mrms_mask = mrms_mask.assign_coords({
    'lat': hrrr_mask.lat,
    'lon': hrrr_mask.lon
})

In [ ]:
total_mask = mrms_mask * hrrr_mask

In [ ]:
total_mask = total_mask.isel(lat=slice(0, -6))

In [ ]:
total_mask = total_mask.assign_coords({'lat': interp.lat.values, 'lon': interp.lon.values})

In [ ]:
total_mask.plot()

## Compute scores

In [ ]:
hrrr = hrrr.isel(lat=slice(0, -6))

In [ ]:
hrrr = hrrr.assign_coords({'lat': interp.lat.values, 'lon': interp.lon.values})

In [ ]:
# Apply mask
mrms = mrms.where(total_mask)
det_pred = det_pred.where(total_mask)
hrrr = hrrr.where(total_mask)
interp = interp.where(total_mask)

In [ ]:
det_pred2 = det_pred2.where(total_mask)


In [ ]:
hrrr.load()

## Bias

In [ ]:
mrms.mean().values

In [ ]:
det_pred.mean().values

In [ ]:
interp.mean().values

In [ ]:
hrrr.mean().values

### Histograms

In [ ]:
bins = np.logspace(0, 2, 25)-1
mid_bin = (bins[1:] + bins[:-1])/2

In [ ]:
def plot_hist(ds, bins, label):
    nums, bins = np.histogram(ds.values, bins=bins)
    plt.plot(mid_bin, nums, marker='o', label=label)

In [ ]:
plt.figure(figsize=(10, 5))
plot_hist(det_pred, bins, 'GAN')
plot_hist(mrms, bins, 'Obs')
plot_hist(interp, bins, 'Interp')
plot_hist(hrrr, bins, 'HRRR')
plt.yscale('log')
plt.legend()

### RMSE

In [ ]:
xs.rmse(det_pred, mrms, dim=['lat', 'lon', 'valid_time'], skipna=True).values

In [ ]:
xs.rmse(interp, mrms, dim=['lat', 'lon', 'valid_time'], skipna=True).values

In [ ]:
xs.rmse(hrrr, mrms, dim=['lat', 'lon', 'valid_time'], skipna=True).values

### FSS

In [ ]:
thresh = 10
window = 100 // 4

In [ ]:
def compute_fss(f, o, thresh, window, time_mean=True):
    f_thresh = f > thresh
    o_thresh = o > thresh
    f_frac = f_thresh.rolling({'lat': window, 'lon': window}, center=True).mean()
    o_frac = o_thresh.rolling({'lat': window, 'lon': window}, center=True).mean()
    mse = ((f_frac - o_frac)**2).mean(('lat', 'lon'))
    mse_ref = (f_frac**2).mean(('lat', 'lon')) + (o_frac**2).mean(('lat', 'lon'))
    fss = 1 - mse / mse_ref
    if time_mean:
        fss = fss.mean('valid_time')
    return fss

In [ ]:
compute_fss(mrms, det_pred, thresh, window).values

In [ ]:
compute_fss(mrms, interp, thresh, window).values

In [ ]:
compute_fss(mrms, hrrr, thresh, window).values

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
det_pred.isel(valid_time=2).plot(vmin=0, vmax=20)
ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
det_pred2.isel(valid_time=2).plot(vmin=0, vmax=20)
ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
mrms.isel(valid_time=2).plot(vmin=0, vmax=20)
ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
interp.isel(valid_time=2).plot(vmin=0, vmax=20)
ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
hrrr.isel(valid_time=2).plot(vmin=0, vmax=20)
ax.set_aspect('equal')

In [ ]:
interp[50].plot(vmin=0, vmax=20)

In [ ]:
hrrr[50].plot(vmin=0, vmax=20)

In [ ]:

 eps = 1e-6
bin_edges = [-eps] + np.linspace(eps, log_retrans(ds_max, tp_log)+eps, 51).tolist()
pred_means.append(np.mean(preds.sel(member=0)))
pred_hists.append(np.histogram(preds.sel(member=0), bins = bin_edges, density=False)[0])
truth_means.append(np.mean(truth))
truth_hists.append(np.histogram(truth, bins = bin_edges, density=False)[0])

truth_pert = truth + np.random.normal(scale=1e-6, size=truth.shape)
preds_pert = preds + np.random.normal(scale=1e-6, size=preds.shape) 
